In [1]:
%load_ext autoreload
%autoreload

import io
import os
import sys
import django
import glob
import json
import re
import requests
from IPython.core.debugger import set_trace
from pathlib import Path
from django.core.files.base import ContentFile
from rest_framework import serializers
from tqdm.notebook import tqdm
from google.cloud import vision
from google.cloud import storage
from PIL import Image, ImageSequence
from gensim.models import Doc2Vec
from IPython.core.debugger import set_trace
import IPython
import collections
from googletrans import Translator
from google.cloud import translate_v2 as translate

parentdir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parentdir)
django.setup()

import getch.models as m
from django.db.models import Q
from django.apps import apps
from django_currentuser.middleware import get_current_user
from datetime import datetime, timedelta

In [2]:
boo = m.Boo.objects.get(pk=355)

In [20]:
rewards = boo.reward_set.all(); rewards

<QuerySet [<Reward: 2021-06-15 몹미까 | quantlab@kakao.com>, <Reward: 2021-06-17 몹미까 | quantlab@kakao.com>, <Reward: 2021-06-18 몹미까 | quantlab@kakao.com>, <Reward: 2021-06-19 몹미까 | quantlab@kakao.com>, <Reward: 2021-06-20 몹미까 | quantlab@kakao.com>, <Reward: 2021-06-21 몹미까 | quantlab@kakao.com>, <Reward: 2021-06-22 몹미까 | quantlab@kakao.com>]>

In [40]:
date_begin = rewards.earliest('date').date; date_begin
# date_end = rewards.last().date

datetime.date(2021, 6, 15)

In [41]:
rewards_out = {}
today = datetime.now().date()

for _r in rewards:
    k = (_r.date-date_begin).days
    rewards_out[k] = {
        'date': str(_r.date),
        'is_today': _r.date == today,
        'n_action': _r.n_collected + _r.n_voted,
        'action_reward': _r.collect_reward + _r.vote_reward,
        'bonus_reward': _r.bonus_reward
    }

rewards_out

{0: {'date': '2021-06-15',
  'is_today': False,
  'n_action': 0,
  'action_reward': 0,
  'bonus_reward': 0},
 2: {'date': '2021-06-17',
  'is_today': False,
  'n_action': 0,
  'action_reward': 0,
  'bonus_reward': 0},
 3: {'date': '2021-06-18',
  'is_today': False,
  'n_action': 226,
  'action_reward': 110,
  'bonus_reward': 0},
 4: {'date': '2021-06-19',
  'is_today': False,
  'n_action': 1,
  'action_reward': 0,
  'bonus_reward': 0},
 5: {'date': '2021-06-20',
  'is_today': False,
  'n_action': 123,
  'action_reward': 60,
  'bonus_reward': 0},
 6: {'date': '2021-06-21',
  'is_today': False,
  'n_action': 69,
  'action_reward': 30,
  'bonus_reward': 0},
 7: {'date': '2021-06-22',
  'is_today': True,
  'n_action': 4,
  'action_reward': 0,
  'bonus_reward': 0}}

In [1]:
from datetime import datetime

In [6]:
datetime.now().date()

datetime.date(2021, 7, 31)

In [21]:
tmp = '''
  <img id='coverpix_preview' src='' style='display:none;height:100px;width:170px;object-fit:cover;' alt='{aa}'>
    <script>
      function handleFileSelect(event) {{
        var files = event.target.files;
        
        for (var i = 0, f; f = files[i]; i++) {{
          if (!f.type.match('image.*')) continue;
          var reader = new FileReader();
          reader.onload = (function () {{
            return function (e) {{
              document.getElementById('coverpix_preview').src = e.target.result;
              document.getElementById('coverpix_preview').style.display = 'block';
            }};
          }})(f);
          reader.readAsDataURL(f);
        }}
      }}
      document.getElementById('id_coverpix').addEventListener('change', handleFileSelect, false);
    </script>'''

tmp.format(aa='test')

"\n  <img id='coverpix_preview' src='' style='display:none;height:100px;width:170px;object-fit:cover;' alt='test'>\n    <script>\n      function handleFileSelect(event) {\n        var files = event.target.files;\n        \n        for (var i = 0, f; f = files[i]; i++) {\n          if (!f.type.match('image.*')) continue;\n          var reader = new FileReader();\n          reader.onload = (function () {\n            return function (e) {\n              document.getElementById('coverpix_preview').src = e.target.result;\n              document.getElementById('coverpix_preview').style.display = 'block';\n            };\n          })(f);\n          reader.readAsDataURL(f);\n        }\n      }\n      document.getElementById('id_coverpix').addEventListener('change', handleFileSelect, false);\n    </script>"

In [10]:
tmp = '''
    <img id='coverpix_preview' src='' style='display:none;height:100px;width:170px;object-fit:cover;' alt='{}'>

'''

tmp.format('a')

"\n    <img id='coverpix_preview' src='' style='display:none;height:100px;width:170px;object-fit:cover;' alt='a'>\n\n"

In [4]:
'abc-ab-c'.replace('-', '_')

'abc_ab_c'

In [15]:
str(datetime.now().timestamp()).replace('.', '_')

'1627738649_052187'

In [4]:
'test' + str(1)

'test1'

In [18]:
['pix_0']

TypeError: 'ResearchItem' object is not subscriptable

In [19]:
getattr(m.ResearchItem.objects.get(pk=7), 'pix_0')

<ImageFieldFile: research/quantlab@kakao.com/2021-07-29/items/2021-08-02_031536.186281__9M_image1.png>

In [33]:
bool(m.ResearchItem.objects.get(pk=7).mcpix_1)

True

In [36]:
not True

False